In [1]:
from fastai.tabular.all import *

comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)

Path.BASE_PATH = path

From the website:

The data for this competition is split into three parts:

- `Train.csv` is the training set, which contains data through the end of 2011.
- `Valid.csv` is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
- `Test.csv` is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

The key fields are in train.csv are:

- `SalesID`: the uniue identifier of the sale
- `MachineID`: the unique identifier of a machine.  A machine can be sold multiple times
- `saleprice`: what the machine sold for at auction (only provided in train.csv)
- `saledate`: the date of the sale

There are several fields towards the end of the file on the different options a machine can have.  The descriptions all start with "machine configuration" in the data dictionary.  Some product types do not have a particular option, so all the records for that option variable will be null for that product type.  Also, some sources do not provide good option and/or hours data.

The machine_appendix.csv file contains the correct year manufactured for a given machine along with the make, model, and product class details. There is one machine id for every machine in all the competition datasets (training, evaluation, etc.).



Considering the lack of information on the columns of the dataset, we have to do some analysis to clean it up.

We use `dtype=object` and `convert_dtypes` so that our dataframe uses proper Pandas extension datatypes (nullable numpy types) for numeric types. Otherwise, as in `auctioneerID`, an integral type would be inferred as a float type due to missing values, which become `nan`. On the other hand, `convert_string=False` is left in since `Categorify` cannot deal with `pd.NA` in `string[python]` datatypes at least; here we have left it as a mixed datatype with `nan` for missing values, which is what `Categorify` expects.

In [2]:
df = pd.read_csv(path/'TrainAndValid.csv', engine='pyarrow', dtype=object).convert_dtypes(convert_string=False)
for colname, col_dtype in df.dtypes.items():
    if col_dtype != np.dtype('O'):
        continue
    df.loc[df[colname] == '', colname] = np.nan
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3,2004,68,Low,11/16/2006 0:00,...,None,None,None,None,None,None,None,None,Standard,Conventional
1,1139248,57000.0,117657,77,121,3,1996,4640,Low,3/26/2004 0:00,...,None,None,None,None,None,None,None,None,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3,2001,2838,High,2/26/2004 0:00,...,None,None,None,None,None,None,None,None,None,None
3,1139251,38500.0,1026470,332,121,3,2001,3486,High,5/19/2011 0:00,...,None,None,None,None,None,None,None,None,None,None
4,1139253,11000.0,1057373,17311,121,3,2007,722,Medium,7/23/2009 0:00,...,None,None,None,None,None,None,None,None,None,None


We also inspect the fields to have a look at the inferred `dtypes`, and the number of filled values. One looks for e.g. ID columns, and ensuries that they have a sensible `dtype`, and sees if the ID are primary-like in the sense that they are mostly non-null.

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  Int64  
 1   SalePrice                 412698 non-null  Float64
 2   MachineID                 412698 non-null  Int64  
 3   ModelID                   412698 non-null  Int64  
 4   datasource                412698 non-null  Int64  
 5   auctioneerID              392562 non-null  Int64  
 6   YearMade                  412698 non-null  Int64  
 7   MachineHoursCurrentMeter  147504 non-null  Int64  
 8   UsageBand                 73670 non-null   object 
 9   saledate                  412698 non-null  object 
 10  fiModelDesc               412698 non-null  object 
 11  fiBaseModel               412698 non-null  object 
 12  fiSecondaryDesc           271971 non-null  object 
 13  fiModelSeries             58667 non-null   o

We also have a look at the number of unique values in each column, and see if there's anything we want to do with low-categorical factors. In addition, also look at the ID's to see if they are dynamic.

In [4]:
for colname, nunique in df.nunique().items():
    if nunique <= 20:
        print(colname, nunique, df[colname].unique())
    else:
        print(colname, nunique)

SalesID 412698
SalePrice 954
MachineID 348808
ModelID 5281
datasource 6 <IntegerArray>
[121, 132, 136, 149, 172, 173]
Length: 6, dtype: Int64
auctioneerID 30
YearMade 73
MachineHoursCurrentMeter 15633
UsageBand 3 ['Low' 'High' 'Medium' None]
saledate 4013
fiModelDesc 5059
fiBaseModel 1961
fiSecondaryDesc 177
fiModelSeries 123
fiModelDescriptor 140
ProductSize 6 [None 'Medium' 'Small' 'Large / Medium' 'Mini' 'Large' 'Compact']
fiProductClassDesc 74
state 53
ProductGroup 6 ['WL' 'SSL' 'TEX' 'BL' 'TTT' 'MG']
ProductGroupDesc 6 ['Wheel Loader' 'Skid Steer Loaders' 'Track Excavators' 'Backhoe Loaders'
 'Track Type Tractors' 'Motor Graders']
Drive_System 4 [None 'Four Wheel Drive' 'Two Wheel Drive' 'No' 'All Wheel Drive']
Enclosure 6 ['EROPS w AC' 'OROPS' 'EROPS' None 'EROPS AC' 'NO ROPS'
 'None or Unspecified']
Forks 2 ['None or Unspecified' None 'Yes']
Pad_Type 4 [None 'None or Unspecified' 'Reversible' 'Street' 'Grouser']
Ride_Control 3 ['None or Unspecified' None 'No' 'Yes']
Stick 2 [Non

For now, maybe we might want to spend some effort tidying up the data, starting from `ProductSize`. Unfortunately, from here onward, especially since we are interacting with the `fastai` library, we should convert them to NumPy datatypes.

We now tidy up `ProductSize`, by making its categorical nature explicit and also by explicating its natural ordering.

In [5]:
sizes = 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact'
df['ProductSize'] = df['ProductSize'].astype('category')
df['ProductSize'].cat.set_categories(sizes, ordered=True)

0            NaN
1         Medium
2            NaN
3          Small
4            NaN
           ...  
412693      Mini
412694      Mini
412695      Mini
412696      Mini
412697      Mini
Name: ProductSize, Length: 412698, dtype: category
Categories (6, object): ['Large' < 'Large / Medium' < 'Medium' < 'Small' < 'Mini' < 'Compact']

We tidy up `SalePrice` by taking its logarithm.

In [6]:
dep_var = 'SalePrice'
df[dep_var] = np.log(df[dep_var])

We tidy up `saledate` with the use of `add_datepart`, to augment with information like holidays, day of week, etc., and then display all the fields related to it.

In [7]:
df['saledate'] = pd.to_datetime(df['saledate'], format='%m/%d/%Y 0:00')

In [8]:
df = add_datepart(df, 'saledate')
' '.join(o for o in df.columns if o.startswith('sale'))

'saleYear saleMonth saleWeek saleDay saleDayofweek saleDayofyear saleIs_month_end saleIs_month_start saleIs_quarter_end saleIs_quarter_start saleIs_year_end saleIs_year_start saleElapsed'

We will perform additional data cleaning later as we convert this dataframe into a `TabularPandas` object that behaves similarly to `Datasets`. We list them here; these are done by `TabularProc`s, which are like `Transform`s; except instead of creating derivatives as the data is accessed, it transforms the data eagerly, and in-place. `Categorify` replaces categorical variables with numerics, and `FillMissing` replaces missing values with the median of the column, then adds a boolean column to mark whether the corresponding value was synthetic.

As for determining which columns are categorical and which continuous, we use the following helper function which implements a simple heuristic.

In [9]:
cont, cat = cont_cat_split(df, 1, dep_var=dep_var)
procs = [Categorify, FillMissing]

The next thing we have to do is determine the training/validation split. Since the objective is to predict future prices, we surmise that we should evaluate a model-creation strategy on its ability to create models that validate well on future data. In this case, that means that we should train a model on data before a certain cutoff and evaluate it on data after it, then train a model with the same hyperparameters on the full training set to obtain our final model. Hence we define the following split:

In [10]:
cond = (df.saleYear < 2011) | (df.saleMonth < 10)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx), list(valid_idx))

Finally, we create the tabular object.

In [11]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

/home/jhanschoo/.local/share/hatch/env/virtual/fai/Rcp_9cR7/fastbook/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/home/jhanschoo/.local/share/hatch/env/virtual/fai/Rcp_9cR7/fastbook/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method 